In [3]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
# import json

In [12]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vince\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Load to Memory & Merge Features and Desc.

In [5]:
games_df = pd.read_csv('games.csv')
metadata_df = pd.read_json('games_metadata.json', lines=True)

In [ ]:
df = pd.merge(games_df, metadata_df, on='app_id', how='inner')

In [11]:
games_df = None
metadata_df = None

In [ ]:
df.to_csv('Altered CSVs/merged_game_data.csv', index=False)

# One Hot Encoding

In [ ]:
df = pd.read_csv('Altered CSVs/merged_game_data.csv')

In [3]:
df['tags']

0        ['Action', 'Adventure', 'Parkour', 'Third Pers...
1                                               ['Action']
2        ['Co-op', 'Stealth', 'Indie', 'Heist', 'Local ...
3        ['Zombies', 'Adventure', 'Survival', 'Action',...
4        ['Roguelike', 'Strategy', 'Tower Defense', 'Pi...
                               ...                        
50867                                                   []
50868                                                   []
50869                                                   []
50870    ['Early Access', 'FPS', 'Action', 'Retro', 'Fi...
50871                                                   []
Name: tags, Length: 50872, dtype: object

In [4]:
# type(df['tags'][0])
    # Tags are stored in a str format*

In [3]:
# Iterate through tags and produce a list of all tags -- with repeats
tags_array = []
for tags in df['tags']:
    for tag in tags.replace('[','').replace(']','').replace('\'','').replace('\"','').split(','):
        tags_array.append(tag)

In [4]:
# Keep only unique tags and Remove Empty Tags
unique_tags = set(tags_array)
unique_tags.remove('')

In [ ]:
# Remove Random Whitespace at the Start of Tags & Produce One-Hot Encoding Columns
for tag in unique_tags:
    if(tag[0]==' '):
        tag = tag[1:]   
    df[tag] = 0

In [ ]:
df

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,...,Competitive,Faith,8-bit Music,Arcade,Relaxing,Lemmings,Emotional,1990s,Beat em up,Shoot Em Up
0,13500,Prince of Persia: Warrior Within™,2008-11-21,True,False,False,Very Positive,84,2199,9.99,...,0,0,0,0,0,0,0,0,0,0
1,22364,BRINK: Agents of Change,2011-08-03,True,False,False,Positive,85,21,2.99,...,0,0,0,0,0,0,0,0,0,0
2,113020,Monaco: What's Yours Is Mine,2013-04-24,True,True,True,Very Positive,92,3722,14.99,...,0,0,0,0,0,0,0,0,0,0
3,226560,Escape Dead Island,2014-11-18,True,False,False,Mixed,61,873,14.99,...,0,0,0,0,0,0,0,0,0,0
4,249050,Dungeon of the ENDLESS™,2014-10-27,True,True,False,Very Positive,88,8784,11.99,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50867,2296380,I Expect You To Die 3: Cog in the Machine,2023-09-28,True,False,False,Very Positive,96,101,22.00,...,0,0,0,0,0,0,0,0,0,0
50868,1272080,PAYDAY 3,2023-09-21,True,False,False,Mostly Negative,38,29458,40.00,...,0,0,0,0,0,0,0,0,0,0
50869,1402110,Eternights,2023-09-11,True,False,False,Very Positive,89,1128,30.00,...,0,0,0,0,0,0,0,0,0,0
50870,2272250,Forgive Me Father 2,2023-10-19,True,False,False,Very Positive,95,82,17.00,...,0,0,0,0,0,0,0,0,0,0


In [10]:
for index, row in df.iterrows():
    for tag in row['tags'].replace('[','').replace(']','').replace('\'','').replace('\"','').split(','):
        tag = tag.strip()
        if tag in unique_tags:
            df.at[index, tag] = 1
df

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,...,Competitive,Faith,8-bit Music,Arcade,Relaxing,Lemmings,Emotional,1990s,Beat em up,Shoot Em Up
0,13500,Prince of Persia: Warrior Within™,2008-11-21,True,False,False,Very Positive,84,2199,9.99,...,0,0,0,0,0,0,0,0,0,0
1,22364,BRINK: Agents of Change,2011-08-03,True,False,False,Positive,85,21,2.99,...,0,0,0,0,0,0,0,0,0,0
2,113020,Monaco: What's Yours Is Mine,2013-04-24,True,True,True,Very Positive,92,3722,14.99,...,0,0,0,0,0,0,0,0,0,0
3,226560,Escape Dead Island,2014-11-18,True,False,False,Mixed,61,873,14.99,...,0,0,0,0,0,0,0,0,0,0
4,249050,Dungeon of the ENDLESS™,2014-10-27,True,True,False,Very Positive,88,8784,11.99,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50867,2296380,I Expect You To Die 3: Cog in the Machine,2023-09-28,True,False,False,Very Positive,96,101,22.00,...,0,0,0,0,0,0,0,0,0,0
50868,1272080,PAYDAY 3,2023-09-21,True,False,False,Mostly Negative,38,29458,40.00,...,0,0,0,0,0,0,0,0,0,0
50869,1402110,Eternights,2023-09-11,True,False,False,Very Positive,89,1128,30.00,...,0,0,0,0,0,0,0,0,0,0
50870,2272250,Forgive Me Father 2,2023-10-19,True,False,False,Very Positive,95,82,17.00,...,0,0,0,0,0,0,0,0,0,0


In [11]:
df.to_csv('Altered CSVs/one_hot_encoded_tags.csv', index=False)

# TFIDF

## Preprocessing & Cleaning

In [4]:
df = pd.read_csv('Altered CSVs/one_hot_encoded_tags.csv')

In [12]:
# Print a list with the names of empty descriptions columns
empty_descriptions = df[df['description'].isnull()]
empty_descriptions_list_name = empty_descriptions['title'].tolist()
empty_descriptions_list_id = empty_descriptions['app_id'].tolist()
empty_descriptions_list_name[0:5], empty_descriptions_list_id[0:5]

(['BRINK: Agents of Change',
  'Men of War: Assault Squad 2 - Deluxe Edition upgrade',
  'Borderlands 2: Headhunter 4: Wedding Day Massacre',
  'Sniper Elite 3 - Camouflage Weapons Pack',
  'The House in Fata Morgana Original Soundtrack'],
 [22364, 271850, 245950, 305181, 458790])

In [5]:
# Replace missing descriptions with the title and tags
df['description'] = df['description'].fillna(df['title'] + ' ' + df['tags'])

In [6]:
# Remove 'tags' column
df = df.drop(columns=['tags'])

In [16]:
# Verify that the descriptions are no longer empty
df[df['description'].isnull()]

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,...,Competitive,Faith,8-bit Music,Arcade,Relaxing,Lemmings,Emotional,1990s,Beat em up,Shoot Em Up


In [21]:
# Show App IDs from array
arr = [22364, 271850, 245950, 305181, 458790]
df[df['app_id'].isin(arr)][['app_id','title','description']]

,app_id,title,description
1,22364,BRINK: Agents of Change,BRINK: Agents of Change ['Action']
7,271850,Men of War: Assault Squad 2 - Deluxe Edition u...,Men of War: Assault Squad 2 - Deluxe Edition u...
18,245950,Borderlands 2: Headhunter 4: Wedding Day Massacre,Borderlands 2: Headhunter 4: Wedding Day Massa...
21,305181,Sniper Elite 3 - Camouflage Weapons Pack,Sniper Elite 3 - Camouflage Weapons Pack ['Adv...
30,458790,The House in Fata Morgana Original Soundtrack,The House in Fata Morgana Original Soundtrack ...


In [7]:
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    return text

In [8]:
# Remove Special Character and Punctuation from Descriptions
df['description'] = df['description'].apply(clean_text)

In [9]:
df['description']

0        enter the dark underworld of prince of persia ...
1                            brink agents of change action
2        monaco whats yours is mine is a single player ...
3        escape dead island is a survivalmystery advent...
4        dungeon of the endless is a roguelike dungeond...
                               ...                        
50867            i expect you to die 3 cog in the machine 
50868                                            payday 3 
50869                                          eternights 
50870    embark on a journey into the darkest nightmare...
50871                                           fatalzone 
Name: description, Length: 50872, dtype: object

In [31]:
print(nltk.data.path)

['C:\\Users\\vince/nltk_data', 'c:\\Users\\vince\\anaconda3\\nltk_data', 'c:\\Users\\vince\\anaconda3\\share\\nltk_data', 'c:\\Users\\vince\\anaconda3\\lib\\nltk_data', 'C:\\Users\\vince\\AppData\\Roaming\\nltk_data', 'C:\\nltk_data', 'D:\\nltk_data', 'E:\\nltk_data']


In [32]:
nltk.data.find('tokenizers/punkt/english.pickle')

FileSystemPathPointer('C:\\Users\\vince\\AppData\\Roaming\\nltk_data\\tokenizers\\punkt\\english.pickle')

In [11]:
df.to_csv('Altered CSVs/before_tokenization.csv', index=False)

In [ ]:
df = pd.read_csv('Altered CSVs/before_tokenization.csv')

In [10]:
df['tokens'] = df['description'].apply(word_tokenize)

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\vince/nltk_data'
    - 'c:\\Users\\vince\\anaconda3\\nltk_data'
    - 'c:\\Users\\vince\\anaconda3\\share\\nltk_data'
    - 'c:\\Users\\vince\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\vince\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [35]:
nltk.data.find('tokenizers/punkt/english.pickle')


FileSystemPathPointer('C:\\Users\\vince\\AppData\\Roaming\\nltk_data\\tokenizers\\punkt\\english.pickle')

In [ ]:
df['tokens'].head()

# F
